Notebook that addresses and solves most common issues encountered with iPyparallel
in handling other modules and user-defined functions

AAG 09/2017

In [2]:
import ipyparallel as ipp
import numpy as np

In [3]:
c = ipp.Client()
dview = c[:]

c.ids

TimeoutError: Hub connection request timed out

## Check the basics work

In [93]:
result = dview.map_sync(lambda x: x**10, range(32))
print(result)

[0, 1, 1024, 59049, 1048576, 9765625, 60466176, 282475249, 1073741824, 3486784401, 10000000000, 25937424601, 61917364224, 137858491849, 289254654976, 576650390625, 1099511627776, 2015993900449, 3570467226624, 6131066257801, 10240000000000, 16679880978201, 26559922791424, 41426511213649, 63403380965376, 95367431640625, 141167095653376, 205891132094649, 296196766695424, 420707233300201, 590490000000000, 819628286980801]


## Handle multiple variables (**args other than the mapped argument)

In [96]:
def simplepower(x, a):
    return (x**10) + a

In [97]:
result = dview.map_sync(simplepower, range(32), np.repeat(3,32) )

print(result)

[3, 4, 1027, 59052, 1048579, 9765628, 60466179, 282475252, 1073741827, 3486784404, 10000000003, 25937424604, 61917364227, 137858491852, 289254654979, 576650390628, 1099511627779, 2015993900452, 3570467226627, 6131066257804, 10240000000003, 16679880978204, 26559922791427, 41426511213652, 63403380965379, 95367431640628, 141167095653379, 205891132094652, 296196766695427, 420707233300204, 590490000000003, 819628286980804]


In [98]:
# does NOT work - power not defined?!?!
result = dview.map_sync(lambda x: simplepower(x,3), range(32) )

print(result)

CompositeError: one or more exceptions from call to method: <lambda>
[0:apply]: NameError: name 'simplepower' is not defined
[1:apply]: NameError: name 'simplepower' is not defined
[2:apply]: NameError: name 'simplepower' is not defined
[3:apply]: NameError: name 'simplepower' is not defined

In [100]:
breakpow = lambda x:simplepower(x, a) # elegant way to provide the parameters in one go to the parallel function

dview.push(dict(a=3)) # send parameters
dview.push(dict(simplepower=simplepower)) # send power

result = dview.map_sync(breakpow, range(32) )
print(result)

[3, 4, 1027, 59052, 1048579, 9765628, 60466179, 282475252, 1073741827, 3486784404, 10000000003, 25937424604, 61917364227, 137858491852, 289254654979, 576650390628, 1099511627779, 2015993900452, 3570467226627, 6131066257804, 10240000000003, 16679880978204, 26559922791427, 41426511213652, 63403380965379, 95367431640628, 141167095653379, 205891132094652, 296196766695427, 420707233300204, 590490000000003, 819628286980804]


## Handle standard modules required by the functions

In [101]:
def nppower(x, a):
    return np.sqrt(x**10) + a

In [103]:
with dview.sync_imports(): # pass the module to the engines
    import numpy as np

breakpow = lambda x:simplepower(x, a) # elegant way to provide the parameters in one go to the parallel function

dview.push(dict(a=3)) # send parameters
dview.push(dict(nppower=nppower)) # send power

result = dview.map_sync(breakpow, range(32) )
print(result)

[3, 4, 1027, 59052, 1048579, 9765628, 60466179, 282475252, 1073741827, 3486784404, 10000000003, 25937424604, 61917364227, 137858491852, 289254654979, 576650390628, 1099511627779, 2015993900452, 3570467226627, 6131066257804, 10240000000003, 16679880978204, 26559922791427, 41426511213652, 63403380965379, 95367431640628, 141167095653379, 205891132094652, 296196766695427, 420707233300204, 590490000000003, 819628286980804]


## Handle custom modules with functions therein

In [1]:
with dview.sync_imports(): # pass the module to the engines
    import numpy as np
    import paratest as tt

breakpow = lambda x:tt.mypower(x, a) # elegant way to provide the parameters in one go to the parallel function

dview.push(dict(a=3)) # send parameters
# dview.push(dict(tt.mypower=tt.mypower)) # send power

result = dview.map_sync(breakpow, range(32) )
print(result)

NameError: name 'dview' is not defined

In [48]:
with dview.sync_imports():
    import test

importing test on engine(s)


In [ ]:
with dview.sync_imports():
    import param_sweep as ps

In [ ]:
dview.execute('import param_sweep')

In [ ]:
import param_sweep as ps

In [ ]:
psa = lambda param:ps.param_sweep_anaqle(param, idparam, model, true_params, scramble, sigmas)

mydict=dict(idparam = 0, model = model, true_params = true_params, scramble = scramble, sigmas = sigmas)
dview.push(mydict)